# Predictive Model for Car Market Value

Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

## Data preparation

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from scipy import sparse

In [2]:
df = pd.read_csv('/datasets/car_data.csv')

In [3]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df.columns = df.columns.str.lower()
df.rename(columns={
    'datecrawled': 'date_crawled',
    'price': 'price',
    'vehicletype': 'vehicle_type',
    'registrationyear': 'registration_year',
    'gearbox': 'gearbox',
    'power': 'power',
    'model': 'model',
    'mileage': 'mileage',
    'registrationmonth': 'registration_month',
    'fueltype': 'fuel_type',
    'brand': 'brand',
    'notrepaired': 'not_repaired',
    'datecreated': 'date_created',
    'numberofpictures': 'number_of_pictures',
    'postalcode': 'postal_code',
    'lastseen': 'last_seen'
}, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   mileage             354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [6]:
# Filling missing values
df['registration_year'].fillna(df['registration_year'].median(), inplace=True)
df['power'].fillna(df['power'].median(), inplace=True)
df['registration_month'].replace(0, np.nan, inplace=True)
df['registration_month'].fillna(df['registration_month'].mode()[0], inplace=True)

# Drop non-informative columns if they exist
if 'number_of_pictures' in df.columns:
    df.drop(columns=['number_of_pictures'], inplace=True)

# Remove unrealistic values
df = df[(df['registration_year'] >= 1920) & (df['registration_year'] <= 2023)]
df = df[df['power'] >= 50]

# Check for and drop duplicates
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")
df.drop_duplicates(inplace=True)

# Final dataset check
print(df.describe())
print(df.info())


Number of duplicate rows: 254
               price  registration_year          power        mileage  \
count  307895.000000      307895.000000  307895.000000  307895.000000   
mean     4760.017191        2003.398688     125.764137  128738.222446   
std      4601.443245           6.765898     198.490993   36566.527367   
min         0.000000        1930.000000      50.000000    5000.000000   
25%      1299.000000        1999.000000      80.000000  125000.000000   
50%      3000.000000        2003.000000     114.000000  150000.000000   
75%      6900.000000        2008.000000     150.000000  150000.000000   
max     20000.000000        2019.000000   20000.000000  150000.000000   

       registration_month    postal_code  
count       307895.000000  307895.000000  
mean             6.151315   51111.378577  
std              3.341854   25793.270531  
min              1.000000    1067.000000  
25%              3.000000   30890.000000  
50%              6.000000   50129.000000  
75%        

In [7]:
df.loc[df['vehicle_type'].isna(), 'vehicle_type'] = 'unknown'
df.loc[df['not_repaired'].isna(), 'not_repaired'] = 'unknown'
df.loc[df['fuel_type'].isna(), 'fuel_type'] = 'unknown'
df.loc[df['gearbox'].isna(), 'gearbox'] = 'unknown'
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307895 entries, 1 to 354368
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   date_crawled        307895 non-null  object 
 1   price               307895 non-null  int64  
 2   vehicle_type        307895 non-null  object 
 3   registration_year   307895 non-null  int64  
 4   gearbox             307895 non-null  object 
 5   power               307895 non-null  int64  
 6   model               295150 non-null  object 
 7   mileage             307895 non-null  int64  
 8   registration_month  307895 non-null  float64
 9   fuel_type           307895 non-null  object 
 10  brand               307895 non-null  object 
 11  not_repaired        307895 non-null  object 
 12  date_created        307895 non-null  object 
 13  postal_code         307895 non-null  int64  
 14  last_seen           307895 non-null  object 
dtypes: float64(1), int64(5), object(9)

In [8]:
# Find and count duplicate rows
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

# Display the duplicate rows if needed
duplicates = df[df.duplicated()]
print(duplicates)

Number of duplicate rows: 0
Empty DataFrame
Columns: [date_crawled, price, vehicle_type, registration_year, gearbox, power, model, mileage, registration_month, fuel_type, brand, not_repaired, date_created, postal_code, last_seen]
Index: []


In [9]:
df.drop(columns=['date_crawled', 'date_created', 'last_seen', 'brand', 'model'], inplace=True)

## Model training

In [10]:
# Convert categorical columns to category dtype
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].astype('category')

# Convert categorical columns to one-hot encoding using sparse matrices
df_sparse = pd.get_dummies(df, sparse=True)

# Ensure all columns are numeric
for col in df_sparse.columns:
    df_sparse[col] = pd.to_numeric(df_sparse[col], errors='coerce')

In [11]:
# Apply variance threshold
threshold = 0.01  
selector = VarianceThreshold(threshold=threshold)
X = df_sparse.drop(columns=['price'])
X = selector.fit_transform(X)
y = df_sparse['price']

# Convert to sparse matrices
X_sparse = sparse.csr_matrix(X)
y_sparse = y.values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sparse, y_sparse, test_size=0.2, random_state=12345)

# Function to calculate RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/utils/validation.py:515: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/utils/validation.py:515: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [12]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
rmse_lr = rmse(y_test, y_pred_lr)
print(f"Linear Regression RMSE: {rmse_lr}")

Linear Regression RMSE: 3283.486995233522


In [13]:
dt_params = {'max_depth': [5], 'min_samples_split': [2]}
dt = GridSearchCV(DecisionTreeRegressor(random_state=12345), dt_params, cv=3, scoring='neg_mean_squared_error')
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
rmse_dt = rmse(y_test, y_pred_dt)
print(f"Decision Tree RMSE: {rmse_dt}")

Decision Tree RMSE: 2562.9680870017874


In [14]:
rf_params = {
    'n_estimators': [10, 50],  
    'max_depth': [5, 10],  
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']  
}
rf = RandomizedSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=3, scoring='neg_mean_squared_error', n_iter=10, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rmse_rf = rmse(y_test, y_pred_rf)
print(f"Random Forest RMSE: {rmse_rf}")


Random Forest RMSE: 2250.7961685733308


In [17]:
# LightGBM
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [18]:
gbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=lgb_eval, early_stopping_rounds=10)
y_pred_gbm = gbm.predict(X_test, num_iteration=gbm.best_iteration)
rmse_gbm = rmse(y_test, y_pred_gbm)
print(f"LightGBM RMSE: {rmse_gbm}")

/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.177479 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's rmse: 4299.21
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 4018.24
[3]	valid_0's rmse: 3773.6
[4]	valid_0's rmse: 3556.53
[5]	valid_0's rmse: 3371.93
[6]	valid_0's rmse: 3206.38
[7]	valid_0's rmse: 3065.23
[8]	valid_0's rmse: 2941.11
[9]	valid_0's rmse: 2828.04
[10]	valid_0's rmse: 2734.65
[11]	valid_0's rmse: 2649.53
[12]	valid_0's rmse: 2575.77
[13]	valid_0's rmse: 2514.13
[14]	valid_0's rmse: 2460.9
[15]	valid_0's rmse: 2413.14
[16]	valid_0's rmse: 2372.66
[17]	valid_0's rmse: 2335.39
[18]	valid_0's rmse: 2303.24
[19]	valid_0's rmse: 2274.93
[20]	valid_0's rmse: 2250.42
[21]	valid_0's rmse: 2225.52
[22]	valid_0's rmse: 2205.88
[23]	valid_0's rmse: 2189.13
[24]	valid_0's rmse: 2172.68
[25]	valid_0's rmse: 2158

Linear Regression RMSE: 3283.49

Decision Tree RMSE: 2562.97

Random Forest RMSE: 2250.80

LightGBM RMSE: 1979.80

It looks like the LightGBM model performed the best with the lowest RMSE, followed by the Random Forest, Decision Tree, and Linear Regression models.

## Model analysis

**Quality Analysis (Based on RMSE)**

**Linear Regression**: RMSE = 3283.49

Linear Regression serves as a baseline model. Its higher RMSE indicates that it doesn’t capture the complexities of the data as well as the other models.

**Decision Tree**: RMSE = 2562.97

The Decision Tree model performs better than Linear Regression, but it may still be prone to overfitting, especially with limited hyperparameter tuning.

**Random Forest**: RMSE = 2250.80

The Random Forest model shows a significant improvement over the Decision Tree, benefiting from the ensemble approach that reduces overfitting and improves generalization.

**LightGBM**: RMSE = 1979.80

LightGBM outperforms all other models, indicating its strong ability to handle complex patterns in the data. It also benefits from efficient handling of large datasets and faster training times.

**Speed Analysis (Based on Training Time)**

**Linear Regression:**

Linear Regression is typically very fast to train, making it suitable for quick sanity checks and baseline comparisons.

**Decision Tree:**

Decision Trees are relatively fast to train, but the speed can vary depending on the depth and complexity of the tree.

**Random Forest:**

Random Forests take longer to train compared to individual Decision Trees due to the ensemble approach. However, they offer better performance and robustness.

**LightGBM:**

LightGBM is designed for high efficiency and speed. It can handle large datasets and complex patterns quickly, making it an excellent choice for this task.

**Summary**

**Quality:** LightGBM has the best performance with the lowest RMSE, followed by Random Forest, Decision Tree, and Linear Regression.

**Speed:** Linear Regression is the fastest to train, followed by Decision Tree, Random Forest, and LightGBM. However, LightGBM’s training time is justified by its superior performance.